# CISC/CMPE 452/COGS 400 Group Project - Distracted Driver Detection

Please put your name and student id

    Jared Taylor, 20075820


1. Create model
2. Train model
3. Test model

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from tensorflow.python.client import device_lib
#from dotenv import load_dotenv

In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
print(device_lib.list_local_devices())

# Model Implementation

In [ ]:
# define data generators
trainDir = 'D:/Documents/Driver Detection/ppdata/train'
testDir = 'D:/Documents/Driver Detection/ppdata/test'

trainDataGen = keras.preprocessing.image.ImageDataGenerator(rescale = 1./255,
                                    rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   zoom_range=0.2,
                                   shear_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')
testDataGen = keras.preprocessing.image.ImageDataGenerator(rescale = 1./255)

trainGenerator = trainDataGen.flow_from_directory(trainDir, batch_size = 32, class_mode = 'categorical', target_size = (112, 112))
testGenerator = testDataGen.flow_from_directory(testDir, batch_size = 32, class_mode = 'categorical', target_size = (112, 112))

In [ ]:
# callback to stop training at certain threshold
class myCallback(keras.callbacks.Callback):
    
    # change cause won't ever happen
    def on_epoch_end(self, epoch, logs = {}):
        if (logs.get('acc') > 0.99 and logs.get('val_acc') > 0.99):
            print('\nCancelling training as model reached 99 accuracy and 99 validation acc.')
            self.model.stop_training == True


In [ ]:
# plot results
def plotResult(history):
    acc = history.history['accuracy']
    valAcc = history.history['val_accuracy']
    loss = history.history['loss']
    valLoss = history.history['val_loss']

    epochs = range(len(acc))

    plt.plot(epochs, acc, 'r', label = 'Training Accuracy')
    plt.plot(epochs, valAcc, 'b', label = 'Validation Accuracy')
    plt.title('Training and Validation Accuracy')
    plt.legend(loc = 0)
    plt.figure()
    plt.show()

# CNN Model

In [ ]:
# CNN model
width, height = 112, 112

inputShape = (width, height, 3)

inputLayer = keras.layers.Input(shape = inputShape)
outputLayer = keras.layers.Dense(10, activation = 'softmax')

x = keras.layers.Conv2D(32, (2, 2), activation = 'relu')(inputLayer)
x = keras.layers.BatchNormalization(axis = 3)(x)
x = keras.layers.MaxPooling2D(pool_size = (2, 2))(x)
x = keras.layers.BatchNormalization()(x)

x = keras.layers.Conv2D(32, (2, 2), activation = 'relu')(x)
x = keras.layers.BatchNormalization(axis = 3)(x)
x = keras.layers.MaxPooling2D(pool_size = (2, 2))(x)
x = keras.layers.BatchNormalization()(x)

x = keras.layers.Conv2D(64, (2, 2), activation = 'relu')(x)
x = keras.layers.BatchNormalization(axis = 3)(x)
x = keras.layers.MaxPooling2D(pool_size = (2, 2))(x)
x = keras.layers.BatchNormalization()(x)

x = keras.layers.Flatten()(x)
x = keras.layers.Dense(1024, activation = 'relu')(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Dropout(0.2)(x)

outputLayer = keras.layers.Dense(10, activation = 'softmax')(x)

#if input_tensor

cnnModel = keras.Model(inputs = inputLayer, outputs = outputLayer)
cnnModel.compile(optimizer = keras.optimizers.RMSprop(lr = 0.0001), loss = 'categorical_crossentropy', metrics = ['accuracy'])
cnnModel.summary()
#keras.utils.plot_model(cnnModel, to_file='cnn_model.png', show_shapes=True, show_layer_names=True)

In [ ]:
# train/fit model
callbacks = myCallback()
history = cnnModel.fit(x = trainGenerator, validation_data = testGenerator, validation_freq = 1, epochs = 10, verbose = 1) #steps_per_epoch = 100

In [ ]:
# plot training and results
print(history.history.keys())
plotResult(history)

# InceptionV3 Model

In [ ]:
# Import InceptionV3 and perform transfer learning
from keras.applications.inception_v3 import InceptionV3
preTrainedModel = InceptionV3(input_shape = (112, 112, 3), include_top = False, weights = None) #weights = 'imagenet'
WEIGHTS_PATH_NO_TOP = (
    'https://github.com/fchollet/deep-learning-models/'
    'releases/download/v0.5/'
    'inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5')
iv3Weights = keras.utils.get_file('inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5', WEIGHTS_PATH_NO_TOP, cache_subdir='models',
                file_hash='bcbd6486424b2319ff4ef7d526e38f63')

preTrainedModel.load_weights(iv3Weights)

for layer in preTrainedModel.layers:
    layer.trainable = False

preTrainedModel.summary()

# add dense layers after pretrained model
lastLayer = preTrainedModel.get_layer('mixed7')
lastOutput = lastLayer.output

x = keras.layers.Flatten()(lastOutput)
x = keras.layers.Dense(1024, activation='relu')(x)
x = keras.layers.Dropout(0.2)(x)
x = keras.layers.Dense(10, activation = 'softmax')(x)

model = keras.Model(preTrainedModel.input, x)
model.compile(optimizer = keras.optimizers.RMSprop(lr = 0.0001), loss = 'categorical_crossentropy', metrics = ['accuracy'])
model.summary()


In [ ]:
# Train/fit InceptionV3 model
inceptionHistory = model.fit(x = trainGenerator, validation_data = testGenerator, validation_freq = 1, epochs = 10, verbose = 1) #30 epochs??

In [ ]:
# plot results
plotResult(inceptionHistory)

In [ ]:
#plot results
'''
        plt.plot(model1.history['train_acc'], label='train_acc')
        plt.plot(model1.history['test_acc'], label='test_acc')
        plt.legend()
        plt.show()
'''


In [ ]:
#predict
#y_pred = model1.predict(x_test)
#evaluator(y_test, y_pred)